In [1]:
import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scprint.tasks import GRNfer
import umap

from grnndata import GRNAnnData, read_h5ad
from grnndata import utils 
from bengrn import BenGRN


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import Counter
from scprint.tasks.grn import get_GTdb


%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

💡 loaded instance: jkobject/scprint


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
2024-03-31 22:03:59,064:INFO - Downloading data from `https://omnipathdb.org/queries/enzsub?format=json`
2024-03-31 22:03:59,064:INFO - Downloading data from `https://omnipathdb.org/queries/enzsub?format=json`
2024-03-31 22:03:59,176:INFO - Downloading data from `https://omnipathdb.org/queries/interactions?format=json`
2024-03-31 22:03:59,264:INFO - Downloading data from `https://omnipathdb.org/queries/complexes?format=json`
2024-03-31 22:03:59,358:INFO - Downloading data from `https://omnipathdb.org/queries/annotations?format=json`
2024-03-31 22:03:59,453:INFO - Downloading data from `https://omnipathdb.org/queries/intercell?format=json`
2024-03-31 22:03:59,624:INFO - Downloading data from `https://omnipathdb.org/about?format=text`


# Load Model and data

In [ ]:
#model = scPrint.load_from_checkpoint('../../data/tensorboard/scprint_test/urxxwu28/checkpoints/epoch=0-step=20000.ckpt') #chocolate-surf-8
model = scPrint.load_from_checkpoint('../../data/temp/cmgiqxw/epoch=3-step=20000.ckpt', precpt_gene_emb = '../../data/temp/embeddings.parquet')
# ../../data/temp/epoch=4-step=12500.ckpt latest model maestro (fortuitous laugther)
# ../../data/logs/scprint_test/f4vao230/checkpoints/epoch=10-step=27500.ckpt looked good as a training run (vermlillion noodle)
# d1792z9k/epoch=3-step=10000.ckpt
# '../../data/temp/d1792z9k/epoch=3-step=10000.ckpt'
# '../../data/temp/6q3dzi3x/epoch=3-step=10000.ckpt'
# '../../data/temp/grsachbg/epoch=2-step=15000.ckpt'
## post debug
# "../lightning_logs/version_14/checkpoints/epoch=1-step=5000.ckpt"

In [ ]:
#only dataset dropped
adata = sc.read_h5ad('/home/ml4ig1/scprint/.lamindb/z7ZZxORUQFshySbuvNW7.h5ad') 

# one that it trained on: z7ZZxORUQFshySbuvNW7.
# untrained on: BljRloq1xjcxRNDpejzI
# and yBCKp6HmXuHa0cZptMo7

In [ ]:
adata = sc.read(
    "data/pancreas_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539828",
)


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [ ]:
adata = sc.read(
    "data/lung_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539942",
)

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [ ]:
preprocessor = Preprocessor(subset_hvg=1000, use_layer='counts', is_symbol=True, force_preprocess=True, skip_validate=True, do_postp=False)

In [ ]:
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
adata = preprocessor(adata.copy())

# Run tests

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="random expr",
    preprocess="softmax",
    head_agg='mean',
    cell_agg="mean"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## most var within

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var within",
    preprocess="softmax",
    head_agg='mean',
    cell_agg="mean"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## most var without

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var without",
    preprocess="softmax",
    head_agg='mean',
    cell_agg="mean"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## max head

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="random expr",
    preprocess="softmax",
    head_agg='max',
    cell_agg="mean"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## consensus

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="random expr",
    preprocess="softmax",
    head_agg='mean',
    cell_agg="consensus"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## sinkhorn

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var without",
    preprocess="sinkhorn",
    head_agg='mean',
    cell_agg="consensus"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## top-k

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var without",
    preprocess="sinkhorn",
    head_agg='mean',
    cell_agg="consensus"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

# mst

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var without",
    preprocess="sinkhorn",
    head_agg='mean',
    cell_agg="consensus"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()

## tmfg

In [ ]:
grn_inferer = GRNfer(model, adata,
    how="most var without",
    preprocess="sinkhorn",
    head_agg='mean',
    cell_agg="consensus"
    filtration="thresh",
    #k=
    #genes=
    #known_grn
)
grn = grn_inferer(cell_type="oligodendrocyte precursor cell", layer=3)

In [ ]:
BenGRN(grn).scprint_benchmark()